In [1]:
import pandas as pd
import os
import zipfile
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from numpy.random import normal 

In [3]:
df=pd.read_excel('gpb_adm_data.xlsx', sheet_name='TIDS')[['TID', 'longitude', 'latitude']].set_index('TID')

In [5]:
pd.DataFrame(columns=['TID', 'latitude', 'longitude', 'link', 'adress', 'metro_station', 'distance_to_station']
            ).to_csv('adm_parsing_data.csv', index=False)
pd.DataFrame(columns=['TID', 'organization_name', 'organization_name']
            ).to_csv('adm_organizations.csv', index=False)

In [14]:
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
def discover(tids, latitude, longtitude):
    link=f"https://yandex.ru/maps/213/moscow/?ll={longtitude}%2C{latitude}&mode=whatshere&whatshere%5Bpoint%5D={longtitude}%2C{latitude}&whatshere%5Bzoom%5D=17.93&z=17.93"
    driver.get(link)
    adress_xpath='//div[@class="toponym-card-title-view__description"]'
    WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, adress_xpath)))
    driver.implicitly_wait(0.4)
    element=driver.find_element(By.XPATH, adress_xpath)
    main_adress=element.text
    print(main_adress)
    transport_xpath='//div[@aria-label="Как добраться"]'
    element=WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.XPATH, transport_xpath)))
    ActionChains(driver).move_to_element(driver.find_element(By.XPATH, transport_xpath)).perform()
    metro=element.find_elements(By.XPATH, '//div[@aria-label="Метро рядом"]')
    if metro!=[]:
        metro_station=metro[0].find_element(By.XPATH, 
                                            '//div[@class="masstransit-stops-view__stop-name"]').text
        metro_distance=metro[0].find_element(By.XPATH, 
                                             '//div[@class="masstransit-stops-view__stop-distance"]').text
        print('\tМетро:', metro_station, metro_distance)
    else:
        metro_station, metro_distance=(None, None)
    pd.DataFrame([[tids, latitude, longtitude, link, main_adress, metro_station, metro_distance]]
            ).to_csv('adm_parsing_data.csv', index=False, header=False, mode='a')
    time_to_wait=normal(1.2, 1.2, 1)[0]
    driver.implicitly_wait(time_to_wait if time_to_wait>0 else 0.1)
    orgaizations_button=driver.find_elements(By.XPATH, 
                                             '//div[contains(@aria-label, "Организации внутри")]')
    if orgaizations_button!=[]:
        orgaizations_button[0].click()
        driver.implicitly_wait(0.4)
        organization_list=driver.find_elements(By.XPATH, 
                '//div[@class="card-businesses-list__list"]//div[@class="search-business-snippet-view"]')
        if organization_list!=[]:
            print("\tОрганизации: ")
        to_frame=[]
        for org in organization_list:
            org_name=org.find_element(By.XPATH, 
                                      './/div[@class="search-business-snippet-view__title"]').text
            org_category=org.find_elements(By.XPATH, 
                                          './/div[@class="search-business-snippet-view__categories"]')
            org_category=None if org_category==[] else org_category[0].text
            print("\t\t", org_name, org_category)
            to_frame.append([tids, org_name, org_category])
        pd.DataFrame(to_frame).to_csv('adm_organizations.csv', index=False, header=False, mode='a')
    time_to_wait=normal(1.2, 1.2, 1)[0]
    driver.implicitly_wait(time_to_wait if time_to_wait>0 else 0.1)


In [16]:
for tids, r in df.iloc[:10].iterrows():
    discover(tids, r['latitude'], r['longitude'])

Котельническая набережная, 29, Москва, 125047
	Метро: Таганская 580 м
	Организации: 
		 Альтера инвест Продажа готового бизнеса и франшиз
		 Арт-галерея Бизнес-центр
		 Luciano Офис продаж
		 НЕФТЕГАЗСНАБ Снабжение строительных объектов
3-й Крутицкий переулок, 11, Москва, 109044
	Метро: Пролетарская 107 м
	Организации: 
		 Torro grill Ресторан
		 Grishko Карнавальные и театральные костюмы
		 Центр обслуживания часов Ремонт часов
		 Invitro Медицинская лаборатория
		 Столовая Столовая
		 Uni-wear Печать на ткани
		 Российский университет медицины ВУЗ
		 Элекснет None
		 2 Проектор Согласование перепланировок
		 Dom.ru Wi-Fi Wi-Fi хот-спот
		 Faceology Салон красоты, косметология
		 Фото-ВидеоЦентр Фотоуслуги, оцифровка
		 Дентал Косметолоджи Стоматологическая клиника
		 Makaroff Обучение мастеров для салонов красоты
		 City Nails Салон красоты
		 Guru_massage Эпиляция, депиляция
		 Пальчики Ногтевая студия, салон бровей и ресниц
		 Шевелюр Парикмахерская, салон красоты
		 ТрансАвто-7 Пу